In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import sys
sys.path.append ('../../')

In [3]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

from pandas.io.json import json_normalize

In [8]:
!mkdir -p {PATH}

In [4]:
PATH = "data/ga-customer-revenue-prediction/"

In [10]:
!ls {PATH}

In [13]:
!export KAGGLE_USERNAME=vandosant && export KAGGLE_KEY=cb365ec3a8405eb7d4224b7575fba73e && kaggle competitions download -c ga-customer-revenue-prediction

  0%|                                               | 0.00/5.22M [00:00<?, ?B/s]
100%|######################################| 5.22M/5.22M [00:00<00:00, 60.1MB/s]
 92%|##################################9   | 49.0M/53.3M [00:01<00:00, 37.4MB/s]
100%|######################################| 53.3M/53.3M [00:01<00:00, 38.7MB/s]
 97%|####################################9 | 56.0M/57.5M [00:00<00:00, 63.6MB/s]
100%|######################################| 57.5M/57.5M [00:01<00:00, 60.3MB/s]


In [14]:
!ls

Ethics in Data Science.ipynb	      lesson4-mnist_sgd.ipynb
bulldozer_dl.ipynb		      lesson5-nlp.ipynb
bulldozer_linreg.ipynb		      marketdata_sample.csv
data				      news_sample.csv
excel				      ppt
ga-customer-revenue-prediction.ipynb  sample_submission.csv.zip
images				      test.csv.zip
kaggle.json			      tmp
lesson1-rf.ipynb		      train.csv.zip
lesson2-rf_interpretation.ipynb       two-sigma.ipynb
lesson3-rf_foundations.ipynb


In [15]:
!mv train.csv.zip test.csv.zip {PATH}

In [16]:
!ls {PATH}

test.csv.zip  train.csv.zip


In [17]:
!unzip {PATH}test.csv.zip

Archive:  data/ga-customer-revenue-prediction/test.csv.zip
  inflating: test.csv                


In [18]:
!unzip {PATH}train.csv.zip

Archive:  data/ga-customer-revenue-prediction/train.csv.zip
  inflating: train.csv               


In [19]:
!ls {PATH}

test.csv.zip  train.csv.zip


In [20]:
!mv test.csv {PATH}

In [21]:
!mv train.csv {PATH}

In [22]:
!ls {PATH}

test.csv  test.csv.zip	train.csv  train.csv.zip


In [5]:
df_raw = pd.read_csv(f'{PATH}train.csv', low_memory=False, parse_dates=['date'])

In [6]:
df_raw.head().T

,0,1,2,3,4
channelGrouping,Organic Search,Organic Search,Organic Search,Organic Search,Organic Search
date,2016-09-02 00:00:00,2016-09-02 00:00:00,2016-09-02 00:00:00,2016-09-02 00:00:00,2016-09-02 00:00:00
device,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...","{""browser"": ""Firefox"", ""browserVersion"": ""not ...","{""browser"": ""Chrome"", ""browserVersion"": ""not a...","{""browser"": ""UC Browser"", ""browserVersion"": ""n...","{""browser"": ""Chrome"", ""browserVersion"": ""not a..."
fullVisitorId,1131660440785968503,377306020877927890,3895546263509774583,4763447161404445595,27294437909732085
geoNetwork,"{""continent"": ""Asia"", ""subContinent"": ""Western...","{""continent"": ""Oceania"", ""subContinent"": ""Aust...","{""continent"": ""Europe"", ""subContinent"": ""South...","{""continent"": ""Asia"", ""subContinent"": ""Southea...","{""continent"": ""Europe"", ""subContinent"": ""North..."
sessionId,1131660440785968503_1472830385,377306020877927890_1472880147,3895546263509774583_1472865386,4763447161404445595_1472881213,27294437909732085_1472822600
socialEngagementType,Not Socially Engaged,Not Socially Engaged,Not Socially Engaged,Not Socially Engaged,Not Socially Engaged
totals,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",..."
trafficSource,"{""campaign"": ""(not set)"", ""source"": ""google"", ...","{""campaign"": ""(not set)"", ""source"": ""google"", ...","{""campaign"": ""(not set)"", ""source"": ""google"", ...","{""campaign"": ""(not set)"", ""source"": ""google"", ...","{""campaign"": ""(not set)"", ""source"": ""google"", ..."
visitId,1472830385,1472880147,1472865386,1472881213,1472822600


In [18]:
with pd.option_context('display.max_rows', 10, 'display.max_columns', 50):
    display(df_raw)

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,2016-09-02,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,1472830385
1,Organic Search,2016-09-02,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,1472880147
2,Organic Search,2016-09-02,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3895546263509774583,"{""continent"": ""Europe"", ""subContinent"": ""South...",3895546263509774583_1472865386,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472865386,1,1472865386
3,Organic Search,2016-09-02,"{""browser"": ""UC Browser"", ""browserVersion"": ""n...",4763447161404445595,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",4763447161404445595_1472881213,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472881213,1,1472881213
4,Organic Search,2016-09-02,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",27294437909732085,"{""continent"": ""Europe"", ""subContinent"": ""North...",27294437909732085_1472822600,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472822600,2,1472822600
...,...,...,...,...,...,...,...,...,...,...,...,...
903648,Social,2017-01-04,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",5123779100307500332,"{""continent"": ""Americas"", ""subContinent"": ""Car...",5123779100307500332_1483554750,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""17"", ""pageviews"": ""15...","{""referralPath"": ""/yt/about/"", ""campaign"": ""(n...",1483554750,1,1483554750
903649,Social,2017-01-04,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",7231728964973959842,"{""continent"": ""Asia"", ""subContinent"": ""Souther...",7231728964973959842_1483543798,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""18"", ""pageviews"": ""13...","{""referralPath"": ""/yt/about/"", ""campaign"": ""(n...",1483543798,1,1483543798
903650,Social,2017-01-04,"{""browser"": ""Android Webview"", ""browserVersion...",5744576632396406899,"{""continent"": ""Asia"", ""subContinent"": ""Eastern...",5744576632396406899_1483526434,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""24"", ""pageviews"": ""21...","{""referralPath"": ""/yt/about/ko/"", ""campaign"": ...",1483526434,1,1483526434
903651,Social,2017-01-04,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",2709355455991750775,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",2709355455991750775_1483592857,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""24"", ""pageviews"": ""22...","{""referralPath"": ""/l.php"", ""campaign"": ""(not s...",1483592857,1,1483592864


In [8]:
df_raw.totals.head(1).T

0    {"visits": "1", "hits": "1", "pageviews": "1",...
Name: totals, dtype: object

In [9]:
import json

In [60]:
df_raw.totals = json_normalize(df_raw.totals.apply(json.loads))

In [27]:
df_raw['transactionRevenue'] = df_raw['totals'].map(lambda x: float(json.loads(x).get('transactionRevenue', 0.0)))

In [28]:
df_raw['transactionRevenue'].sum()

1540071240000.0

### Save to file

In [29]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/ga-customer-revenue-prediction')

### Open feather data

In [5]:
df_raw = pd.read_feather('tmp/ga-customer-revenue-prediction')

In [12]:
df_raw.sort_values(by=['transactionRevenue'], ascending=False).head()

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime,transactionRevenue
375253,Display,2017-04-05,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",1957458976293878100,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",1957458976293878100_1491423580,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""58"", ""pageviews"": ""40...","{""campaign"": ""(not set)"", ""source"": ""dfa"", ""me...",1491423580,243,1491423580,2.312950e+10
430822,Direct,2017-02-14,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",1957458976293878100,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",1957458976293878100_1487097028,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""91"", ""pageviews"": ""64...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1487097028,181,1487097028,1.785550e+10
380988,Direct,2016-09-16,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",5632276788326171571,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",5632276788326171571_1474035643,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""136"", ""pageviews"": ""8...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1474035643,5,1474035643,1.602375e+10
773715,Direct,2017-07-18,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9417857471295131045,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",9417857471295131045_1500404409,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""41"", ""pageviews"": ""30...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1500404409,8,1500404409,1.058914e+10
891576,Display,2017-03-24,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",1957458976293878100,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",1957458976293878100_1490380560,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""50"", ""pageviews"": ""39...","{""campaign"": ""(not set)"", ""source"": ""dfa"", ""me...",1490380560,237,1490380560,8.677830e+09


In [14]:
df_raw['transactionRevenueLog'] = np.log(df_raw.transactionRevenue)

/opt/conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


In [16]:
add_datepart(df_raw, 'date')

In [19]:
df_raw.Year.head()

0    2016
1    2016
2    2016
3    2016
4    2016
Name: Year, dtype: int64

In [20]:
train_cats(df_raw)

In [21]:
df_raw.head()

,channelGrouping,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,Organic Search,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,...,2,4,246,False,False,False,False,False,False,1472774400
1,Organic Search,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,...,2,4,246,False,False,False,False,False,False,1472774400
2,Organic Search,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3895546263509774583,"{""continent"": ""Europe"", ""subContinent"": ""South...",3895546263509774583_1472865386,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472865386,1,...,2,4,246,False,False,False,False,False,False,1472774400
3,Organic Search,"{""browser"": ""UC Browser"", ""browserVersion"": ""n...",4763447161404445595,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",4763447161404445595_1472881213,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472881213,1,...,2,4,246,False,False,False,False,False,False,1472774400
4,Organic Search,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",27294437909732085,"{""continent"": ""Europe"", ""subContinent"": ""North...",27294437909732085_1472822600,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472822600,2,...,2,4,246,False,False,False,False,False,False,1472774400


In [23]:
df_raw.isnull().sum().sort_index()/len(df_raw)

Day                      0.0
Dayofweek                0.0
Dayofyear                0.0
Elapsed                  0.0
Is_month_end             0.0
Is_month_start           0.0
Is_quarter_end           0.0
Is_quarter_start         0.0
Is_year_end              0.0
Is_year_start            0.0
Month                    0.0
Week                     0.0
Year                     0.0
channelGrouping          0.0
device                   0.0
fullVisitorId            0.0
geoNetwork               0.0
sessionId                0.0
socialEngagementType     0.0
totals                   0.0
trafficSource            0.0
transactionRevenue       0.0
transactionRevenueLog    0.0
visitId                  0.0
visitNumber              0.0
visitStartTime           0.0
dtype: float64

In [24]:
df_raw.head()

,channelGrouping,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,...,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,Organic Search,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,...,2,4,246,False,False,False,False,False,False,1472774400
1,Organic Search,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,...,2,4,246,False,False,False,False,False,False,1472774400
2,Organic Search,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3895546263509774583,"{""continent"": ""Europe"", ""subContinent"": ""South...",3895546263509774583_1472865386,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472865386,1,...,2,4,246,False,False,False,False,False,False,1472774400
3,Organic Search,"{""browser"": ""UC Browser"", ""browserVersion"": ""n...",4763447161404445595,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",4763447161404445595_1472881213,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472881213,1,...,2,4,246,False,False,False,False,False,False,1472774400
4,Organic Search,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",27294437909732085,"{""continent"": ""Europe"", ""subContinent"": ""North...",27294437909732085_1472822600,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472822600,2,...,2,4,246,False,False,False,False,False,False,1472774400


In [26]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/ga-customer-revenue-prediction')

### Pre-processing

In [6]:
df_raw = pd.read_feather('tmp/ga-customer-revenue-prediction')

In [23]:
json.loads(df_raw['device'][0]).keys()

dict_keys(['browser', 'browserVersion', 'browserSize', 'operatingSystem', 'operatingSystemVersion', 'isMobile', 'mobileDeviceBranding', 'mobileDeviceModel', 'mobileInputSelector', 'mobileDeviceInfo', 'mobileDeviceMarketingName', 'flashVersion', 'language', 'screenColors', 'screenResolution', 'deviceCategory'])

In [28]:
for k in json.loads(df_raw['totals'][0]).keys():
    print(k)
    df_raw[k] = df_raw['totals'].map(lambda x: int(json.loads(x).get(k, 0)))

visits
hits
pageviews
bounces
newVisits


In [26]:
for k in json.loads(df_raw['device'][0]).keys():
    print(k)
    df_raw[k] = df_raw['device'].map(lambda x: json.loads(x).get(k))

browser
browserVersion
browserSize
operatingSystem
operatingSystemVersion
isMobile
mobileDeviceBranding
mobileDeviceModel
mobileInputSelector
mobileDeviceInfo
mobileDeviceMarketingName
flashVersion
language
screenColors
screenResolution
deviceCategory


In [36]:
for k in json.loads(df_raw['trafficSource'][0]).keys():
    print(k)
    if k == 'adwordsClickInfo':
        continue
    else:
        df_raw[k] = df_raw['trafficSource'].map(lambda x: json.loads(x).get(k))

campaign
source
medium
keyword
adwordsClickInfo


In [38]:
for k in json.loads(df_raw['geoNetwork'][0]).keys():
    print(k)
    df_raw[k] = df_raw['geoNetwork'].map(lambda x: json.loads(x).get(k))

continent
subContinent
country
region
metro
city
cityId
networkDomain
latitude
longitude
networkLocation


In [39]:
df_raw.head()

,channelGrouping,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,...,subContinent,country,region,metro,city,cityId,networkDomain,latitude,longitude,networkLocation
0,Organic Search,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,...,Western Asia,Turkey,Izmir,(not set),Izmir,not available in demo dataset,ttnet.com.tr,not available in demo dataset,not available in demo dataset,not available in demo dataset
1,Organic Search,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,...,Australasia,Australia,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,dodo.net.au,not available in demo dataset,not available in demo dataset,not available in demo dataset
2,Organic Search,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3895546263509774583,"{""continent"": ""Europe"", ""subContinent"": ""South...",3895546263509774583_1472865386,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472865386,1,...,Southern Europe,Spain,Community of Madrid,(not set),Madrid,not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset
3,Organic Search,"{""browser"": ""UC Browser"", ""browserVersion"": ""n...",4763447161404445595,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",4763447161404445595_1472881213,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472881213,1,...,Southeast Asia,Indonesia,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset
4,Organic Search,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",27294437909732085,"{""continent"": ""Europe"", ""subContinent"": ""North...",27294437909732085_1472822600,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472822600,2,...,Northern Europe,United Kingdom,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset


In [41]:
without_json = df_raw.drop(['device', 'totals', 'trafficSource', 'geoNetwork'], axis=1)

In [53]:
without_json.head()

,channelGrouping,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,transactionRevenue,transactionRevenueLog,Year,...,subContinent,country,region,metro,city,cityId,networkDomain,latitude,longitude,networkLocation
0,Organic Search,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,0.0,-inf,2016,...,Western Asia,Turkey,Izmir,(not set),Izmir,not available in demo dataset,ttnet.com.tr,not available in demo dataset,not available in demo dataset,not available in demo dataset
1,Organic Search,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,0.0,-inf,2016,...,Australasia,Australia,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,dodo.net.au,not available in demo dataset,not available in demo dataset,not available in demo dataset
2,Organic Search,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,0.0,-inf,2016,...,Southern Europe,Spain,Community of Madrid,(not set),Madrid,not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset
3,Organic Search,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,0.0,-inf,2016,...,Southeast Asia,Indonesia,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset
4,Organic Search,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,0.0,-inf,2016,...,Northern Europe,United Kingdom,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset


In [55]:
without_json.drop(['transactionRevenueLog'], axis=1, inplace=True)

In [56]:
train_cats(without_json)

In [57]:
df, y, nas = proc_df(without_json, 'transactionRevenue')

In [58]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df, y)
m.score(df,y)

0.7406146313106007

In [59]:
os.makedirs('tmp', exist_ok=True)
without_json.to_feather('tmp/ga-customer-revenue-prediction')